In [1]:
from db.connect import connect
con = connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44), 64-bit',)


In [7]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_sql_query('select * from "atendimentos"',con=con)
tipo = 'SP'
ultimo_registro = df[df["tipo_senha"] == tipo]

In [9]:
ultimo_registro = ultimo_registro.sort_values(by = "data_emissao")

,id,tipo_senha,numeracao,codigo_senha,data_emissao,guiche,data_atendimento
2,26,SP,9,SP009,2022-09-06 08:44:17.792000,2.0,2022-09-06 09:07:33.401000
4,32,SP,5,SP005,2022-09-06 09:55:34.518000,5.0,2022-09-06 10:07:40.222000
1,25,SP,8,SP008,2022-09-06 11:43:22.651000,1.0,2022-09-06 12:07:26.636000
7,23,SP,6,SP006,2022-09-06 12:41:44.712000,2.0,2022-09-06 12:45:36.857000
6,22,SP,5,SP005,2022-09-06 13:41:42.311000,2.0,2022-09-06 13:44:43.830000
0,24,SP,7,SP007,2022-09-06 16:41:47.444000,1.0,2022-09-06 17:07:19.874000
32,15,SP,1,SP001,2022-09-07 10:12:53.247000,2.0,2022-09-07 10:13:03.630000
34,33,SP,1,SP001,2022-09-07 10:43:47.199000,1.0,2022-09-07 10:48:51.257000
35,21,SP,4,SP004,2022-09-07 11:30:33.132000,1.0,2022-09-07 11:51:08.919000
36,18,SP,1,SP001,2022-09-07 12:23:13.167000,2.0,2022-09-07 12:50:28.798000


In [10]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_sql_query('select * from "atendimentos"',con=con)
ultimo_registro = df[df["tipo_senha"] == "SP"].tail(1)
ultimo_registro
data_ultimo_registro = pd.to_datetime(ultimo_registro["data_emissao"]).values[0]
data_ultimo_registro

numpy.datetime64('2022-09-07T01:12:53.247825000')

In [19]:
from datetime import datetime, timedelta

Timestamp('2022-09-07 01:12:53.247825+0000', tz='UTC')

In [30]:
pd.Timestamp(data_ultimo_registro).replace(tzinfo=datetime.timezone.utc).astimezone(pytz.timezone('America/Recife'))

Timestamp('2022-09-06 22:12:53.247825-0300', tz='America/Recife')

In [26]:
import pytz

In [32]:
datetime.datetime.now(pytz.timezone('America/Recife'))

datetime.datetime(2022, 9, 6, 22, 40, 47, 614611, tzinfo=<DstTzInfo 'America/Recife' -03-1 day, 21:00:00 STD>)

In [12]:
pd.to_datetime(datetime.datetime.now(datetime.timezone.utc).replace(hour = 7, minute = 0, second = 0, microsecond = 0))

Timestamp('2022-09-07 07:00:00+0000', tz='UTC')

In [3]:
import requests
import random
import time
import tqdm

env = ['http://localhost:8000/', 'https://ticketprint.herokuapp.com/']

env = env[1]

opcoes = ['SG','SP','SE']

for i in tqdm.tqdm(range(0,10)):
    index = random.randint(0,2)
    url = env + "senha/" + opcoes[index]
    r = requests.get(url)
    time.sleep(index)

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


In [1]:
import requests
import random
import time
import tqdm

env = ['http://localhost:8000/', 'https://ticketprint.herokuapp.com/']

env = env[1]

for i in range(0,11):
    guiche = random.randint(1,5)
    url = env + "chamada/" + str(guiche)
    r = requests.get(url)
    print(r.json()) 
    time.sleep(2)

{'senha': 'SE001', 'data_atendimento': '06/09/2022, 23:07:16', 'guiche': '5'}
{'senha': 'SP007', 'data_atendimento': '06/09/2022, 23:07:20', 'guiche': '1'}
{'senha': 'SE002', 'data_atendimento': '06/09/2022, 23:07:23', 'guiche': '3'}
{'senha': 'SP008', 'data_atendimento': '06/09/2022, 23:07:26', 'guiche': '1'}
{'senha': 'SE003', 'data_atendimento': '06/09/2022, 23:07:30', 'guiche': '1'}
{'senha': 'SP009', 'data_atendimento': '06/09/2022, 23:07:33', 'guiche': '2'}
{'senha': 'SE004', 'data_atendimento': '06/09/2022, 23:07:37', 'guiche': '2'}
{'senha': 'SP005', 'data_atendimento': '06/09/2022, 23:07:40', 'guiche': '5'}
{'senha': 'SE005', 'data_atendimento': '06/09/2022, 23:07:43', 'guiche': '5'}
{'senha': 'Não há senhas a serem chamadas'}
{'senha': 'Não há senhas a serem chamadas'}
